# Playing around with some Portfolio Management evaluation ideas

### Imports

In [3]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Common imports
import numpy as np
import pandas as pd
import scipy.stats as stats
import os

# To draw markdown inside of code cells:
from IPython.display import Markdown as md

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"


# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

### Random seed

In [4]:
# to make this notebook's output stable across runs
np.random.seed(42)

## Constants for the remainder

In [39]:
MONTH_DAYS = 21
D = 2 * 12 * MONTH_DAYS
MU = 0.0005
SIGMA = 0.008

In [40]:
# Instead of setting the cell to Markdown, create Markdown from withnin a code cell!
# We can just use python variable replacement syntax to make the text dynamic
md("""## Start with a uniform random variable centered at MU={} and SIGMA={} for a period of {} days

*Though results may vary depending on the seed!!!!*
""".format(MU, SIGMA, D))

## Start with a uniform random variable centered at MU=0.0005 and SIGMA=0.008 for a period of 504 days

*Though results may vary depending on the seed!!!!*


In [41]:
x = np.random.rand(D,1)

In [46]:
days = np.arange(1,D+1)
days = np.reshape(days, (D,1))

months, years = (days-1) // MONTH_DAYS + 1, (days-1) // (12*MONTH_DAYS) + 1, 

x = np.random.rand(D,1)
x = (x - x.mean())/x.std() * SIGMA + MU
x = np.concatenate((days, months, years, x), axis=1)

daily_outperforms = pd.DataFrame(
    {'day': x[:, 0], 
     'month': x[:, 1],
     'year': x[:, 2],
     'outperformance': x[:, 3],
    })
daily_outperforms.describe()

,day,month,year,outperformance
count,504.000000,504.000000,504.000000,504.000000
mean,252.500000,12.500000,1.500000,0.000500
std,145.636534,6.929064,0.500497,0.008008
min,1.000000,1.000000,1.000000,-0.012764
25%,126.750000,6.750000,1.000000,-0.006473
50%,252.500000,12.500000,1.500000,0.000539
75%,378.250000,18.250000,2.000000,0.006596
max,504.000000,24.000000,2.000000,0.015056


In [47]:
daily_outperforms

,day,month,year,outperformance
0,1.0,1.0,1.0,-0.001132
1,2.0,1.0,1.0,-0.008297
2,3.0,1.0,1.0,-0.007998
3,4.0,1.0,1.0,-0.001105
4,5.0,1.0,1.0,0.008351
5,6.0,1.0,1.0,0.012274
6,7.0,1.0,1.0,-0.010419
7,8.0,1.0,1.0,-0.001804
8,9.0,1.0,1.0,-0.009970
9,10.0,1.0,1.0,-0.012302


In [48]:
def get_mu_sigma(outperforms):
    x = outperforms["outperformance"]
    return [x.mean(), x.std()]

In [49]:
mu_daily, sigma_daily = get_mu_sigma(daily_outperforms)
mu_daily, sigma_daily

(0.0005000000000000019, 0.00800794833777772)

In [50]:
daily_prob_value_add = stats.norm.sf(0, loc=mu_daily, scale=sigma_daily)
daily_prob_value_add

0.524892968984184

In [69]:
do_gb = daily_outperforms.drop(labels=['day'], axis=1)
do_gb = do_gb.groupby(['month'])
monthly_outperforms = do_gb.aggregate(np.mean)
monthly_outperforms.describe()

,year,outperformance
count,24.000000,24.000000
mean,1.500000,0.000500
std,0.510754,0.001644
min,1.000000,-0.002498
25%,1.000000,-0.000766
50%,1.500000,0.000619
75%,2.000000,0.001855
max,2.000000,0.003613


In [76]:
monthly_outperforms

,year,outperformance
month,,
1.0,1.0,-0.000719
2.0,1.0,-0.001050
3.0,1.0,-0.000339
4.0,1.0,-0.000942
5.0,1.0,0.003613
6.0,1.0,0.002114
7.0,1.0,0.002265
8.0,1.0,-0.002010
9.0,1.0,0.001840


In [77]:
mu_monthly, sigma_monthly = get_mu_sigma(monthly_outperforms)
mu_monthly, sigma_monthly

(0.0005000000000000013, 0.001644477572056243)

In [78]:
monthly_prob_value_add = stats.norm.sf(0, loc=mu_monthly, scale=sigma_monthly)
monthly_prob_value_add

0.6194543117404713

In [79]:
do_gb = daily_outperforms.drop(labels=['day', 'month'], axis=1)
do_gb = do_gb.groupby(['year'])
yearly_outperforms = do_gb.aggregate(np.mean)
yearly_outperforms.describe()

,outperformance
count,2.000000
mean,0.000500
std,0.000045
min,0.000468
25%,0.000484
50%,0.000500
75%,0.000516
max,0.000532


In [80]:
yearly_outperforms

,outperformance
year,
1.0,0.000468
2.0,0.000532


In [84]:
mu_yearly, sigma_yearly = get_mu_sigma(yearly_outperforms)
mu_yearly, sigma_yearly

(0.0005000000000000013, 4.5422333705244596e-05)

In [85]:
yearly_prob_value_add = stats.norm.sf(0, loc=mu_yearly, scale=sigma_yearly)
yearly_prob_value_add

1.0

In [87]:
.008008346376813566 / np.sqrt(MONTH_DAYS), .008008346376813566 / np.sqrt(MONTH_DAYS * 12)

(0.00174756445062731, 0.000504478402997949)